In [3]:
# Core
import numpy as np
import pandas as pd

In [1]:
# Local imports
import kepler_sieve
from planets_interp import get_earth_pos
from asteroid_element import get_asteroids
from db_config import db_engine
from db_utils import sp2df

In [4]:
ts = np.array([59000])

In [5]:
get_earth_pos(ts)

array([[-3.56168772e-01, -9.44135316e-01,  9.99339693e-05]])